In [2]:
from biopandas.pdb import PandasPdb
from pypdb import *
# from pypdb.clients.pdb.pdb_client import * # Currently does not work due to weird glitch

In [3]:
import numpy

In [4]:
ppdb = PandasPdb().fetch_pdb(uniprot_id='Q5VSL9', source='alphafold2-v2')

In [5]:
ppdb

In [42]:
def pdb_list_txt_file_to_list(filename):
    pdbs = []
    with open(filename) as f:
        for line in f:
            pdbs.append(line[:4])
    return pdbs

In [43]:
casf_pdbs = pdb_list_txt_file_to_list('casf_2016_ids.txt')

In [8]:
def get_all_pdb_info(pdb_id):
    return get_info(pdb_id)

In [9]:
NON_LIGAND_HETATMS = ["F3F", "PJE","DMS", "CL", "MG", "EDO", "PEG", "MLI", "NI", 
                      "SO4", "WAT", "LI", "NA", "K", "RB", "BE", "MG", "CA", "SR",
                      "F", "CL", "BR", "I", "MN", "FE", "CO", "NI", "CU", "ZN", "3CO", 
                      "BUQ", "NAG", "NAD", "CR", "SF4", "EOH", "ZNO", "NAO", "EOM", 
                      "EHN", "NAP", "CCN", "NAW", "BR", "EGL", "NI2", "GSH", "NI1", 
                      "O2", "BA", "RU", "SAH", "GU7", "SAM", "TAS", "DCE", "2BM", 
                      "TP7", "OF3", "OF1", "RB", "IOH", "MW1", "IOD", "C2O", "BNZ",
                      "TCN", "ARS", "NH4", "GD", "PER", "GA", "TPP", "CHX", "CME",
                      "THB", "IPA", "CD1", "OH", "SO4", "DTT", "PQN", "CYN", "PQQ",
                      "PYJ", "PEO", "NA6", "MBR", "NA5", "OS", "MAN", "CMO", "OCL",
                      "DMF", "OCN", "MO3", "NGN", "ACT", "U1", "HDZ", "MO5", "MO4", 
                      "VO4", "DMS", "FUC", "PCL", "CB5", "EEE", "HG", "NO2", "CMP",
                      "PR", "BMA", "IUM", "PT", "ZN2", "TTP", "NO3", "YT3", "TYS",
                      "PB", "M2M", "ZO3", "PD", "AMP", "PI", "MH3", "AF3", "ZN", "MN3",
                      "OXY", "NI", "CSD", "OX", "PS5", "MN5", "MN6", "S", "W", "SB",
                      "FOL", "OXE", "PT4", "PBM", "O", "MW2", "MG", "543", "MSM",
                      "C5P", "ANL", "MTO", "NO", "TBU", "OPY", "PC4", "GU3", "GU2",
                      "GU1", "MOH", "ANP", "GU6", "GU5", "GU4", "AU", "OC3", "BTN",
                      "I42", "OC4", "OC7", "OC6", "TMP", "RE", "GD3", "CTP", "ACE",
                      "3OF", "ETZ", "MM4", "IN", "ACN", "DOD", "AST", "COA", "EU",
                      "DOX", "COB", "B12", "REO", "ATP", "CD3", "U10", "ACY", "PEG", 
                      "YB", "NDP", "NBZ", "ETI", "SER", "C2C", "NA", "FMT", "ASC",
                      "AU3", "FE2", "LNK", "SEK", "MO1", "EU3", "1BO", "AUC", "CLO",
                      "FE", "DUM", "ADP", "OF2", "BEF", "FEL", "BF4", "HEX", "CUZ",
                      "NDG", "XE", "FMN", "YAN", "CUA", "V", "CUO", "HEM", "GMP",
                      "CU", "MGF", "GDP", "CFT", "SBT", "PLP", "SR", "FU1", "EDN", 
                      "EDO", "H2S", "ND4", "BRO", "KR", "CS", "NME", "CDP", "HGI",
                      "SM", "ALY", "NMO", "TDP", "SE", "HO", "3CN", "AZI", "F42",
                      "FLO", "6MO", "EMC", "Y1", "MO7", "SE4", "BF2", "CO", "NGD",
                      "2MO", "202", "DIS", "MBN", "LA", "PGO", "CL", "HP6", "SO2", 
                      "LI", "PPS", "TPO", "POL", "GU0", "SGM", "DTU", "MOO", "TE",
                      "TB", "CA", "FAD", "CNV", "GOL", "SCN", "AG", "PO4", "IR",
                      "DIO", "NH2", "8CL", "3NI", "IRI", "UTP", "AR", "N4M", "CE",
                      "NH3", "MN", "CNN", "HGC", "GU8", "GTP", "UDP", "OC2", "ART",
                      "TFH", "MCH", "2NO", "6WO", "CD5", "KCX", "E1H", "ARF", "TL",
                      "DXE", "GU9", "IDO", "KO4", "NRU", "4MO", "HOH", "010",
                     "PE5",
                      "IMD",
                      "MPD",
                      "PCA",
                      "21H",
                      "LLP",
                      "GOA",
                      "PE4",
                      'PG4',
                      '1PE',
                      'BOG',
                      'TRS',
                     ]


def check_if_apo(pdb_id):
        pdb_file = get_pdb_file(pdb_id, filetype='pdb', compression=False)
        cleaned_pdb_file = list(filter(('').__ne__, pdb_file.split(' ')))
        raw_coord = [i for i, j in enumerate(cleaned_pdb_file) if j == "\nHETNAM" or j =="\nHETSYN"]
        ligs = []
        for i in [a + 1 for a in raw_coord]:
            if cleaned_pdb_file[i] not in NON_LIGAND_HETATMS:
                if len(cleaned_pdb_file[i]) < 3:
                    if cleaned_pdb_file[i+1] not in NON_LIGAND_HETATMS:
                        ligs.append(cleaned_pdb_file[i+1])
                else:
                    ligs.append(cleaned_pdb_file[i])
        if len(ligs) > 0:
            return False
        else:
            return True

In [10]:
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [11]:
def obtain_related_pdb_ids(pdb_id, dict_):
    all_info = get_all_pdb_info(pdb_id)
    try:
        if not all_info:
            pass
        else:
            for i in range(len(all_info['pdbx_database_related'])):
                new_pdb = all_info['pdbx_database_related'][i]['db_id']
                is_apo = check_if_apo(new_pdb)
                if is_apo:
                   append_value(dict_, pdb_id, new_pdb)
        return dict_
    except:
        return dict_

In [12]:
check_if_apo('2DHQ')

/data/pegasus/not-backed-up/durant/miniconda3/envs/pointvs/lib/python3.8/site-packages/pypdb/pypdb.py:486: DeprecationWarning: The `get_pdb_file` function within pypdb.py is deprecated.See `pypdb/clients/pdb/pdb_client.py` for a near-identical function to use
  warnings.warn(


Sending GET request to https://files.rcsb.org/download/2DHQ.pdb to fetch 2DHQ's pdb file as a string.


True

In [25]:
for c in casf_pdbs:
    if check_if_apo(c):
        print('interesting')

# ALPHAFOLD GRABBER

In [80]:
from biopandas.pdb import PandasPdb
import pandas as pd
from pymol import cmd
import warnings
warnings.filterwarnings('ignore')
import urllib

In [81]:
pdb_to_uniprot_csv = pd.read_csv('pdb_chain_uniprot.csv', header=0, skiprows=1)

In [82]:
def pdb_to_uniprot(pdb_id, uniprot_csv):
    raw_data = uniprot_csv[uniprot_csv['PDB'] == pdb_id.lower()][['SP_PRIMARY', 'CHAIN']].values
    chain_uniprot_id_dict = {}
    for i in raw_data:
        chain_uniprot_id_dict[i[1]] = i[0]
    return chain_uniprot_id_dict

In [103]:
def scrape_uniprot_for_sequences(pdb_id, uniprot_csv):
    uniprot_id_dict = pdb_to_uniprot(pdb_id, uniprot_csv)
    uniprot_data = []
    for i in uniprot_id_dict.keys():
        print(uniprot_id_dict[i])
        raw_fasta = str(urllib.request.urlopen("http://www.uniprot.org/uniprot/" + uniprot_id_dict[i] + ".fasta").read())
        sequence_only = ''.join(raw_fasta.split('\\n')[1:])
        uniprot_data.append(sequence_only)
    return uniprot_data


In [105]:
def scrape_rcsb_for_sequence(pdb_id):
    pdb_fasta = urllib.request.urlopen(f'https://www.rcsb.org/fasta/entry/{pdb_id}').read()
    return pdb_fasta

In [104]:
scrape_uniprot_for_sequences('1nc1', pdb_to_uniprot_csv)

P0AF12
P0AF12


['MKIGIIGAMEEEVTLLRDKIENRQTISLGGCEIYTGQLNGTEVALLKSGIGKVAAALGATLLLEHCKPDVIINTGSAGGLAPTLKVGDIVVSDEARYHDADVTAFGYEYGQLPGCPAGFKADDKLIAAAEACIAELNLNAVRGLIVSGDAFINGSVGLAKIRHNFPQAIAVEMEATAIAHVCHNFNVPFVVVRAISDVADQQSHLSFDEFLAVAAKQSSLMVESLVQKLAHG"',
 'MKIGIIGAMEEEVTLLRDKIENRQTISLGGCEIYTGQLNGTEVALLKSGIGKVAAALGATLLLEHCKPDVIINTGSAGGLAPTLKVGDIVVSDEARYHDADVTAFGYEYGQLPGCPAGFKADDKLIAAAEACIAELNLNAVRGLIVSGDAFINGSVGLAKIRHNFPQAIAVEMEATAIAHVCHNFNVPFVVVRAISDVADQQSHLSFDEFLAVAAKQSSLMVESLVQKLAHG"']

In [106]:
scrape_rcsb_for_sequence('1nc1')

b'>1NC1_1|Chains A, B|MTA/SAH nucleosidase|Escherichia coli (562)\nFQGAMDPEFSMKIGIIGAMEEEVTLLRDKIENRQTISLGGCEIYTGQLNGTEVALLKSGIGKVAAALGATLLLEHCKPDVIINTGSAGGLAPTLKVGDIVVSDEARYHDADVTAFGYEYGQLPGCPAGFKADDKLIAAAEACIAELNLNAVRGLIVSGDAFINGSVGLAKIRHNFPQAIAVEMEATAIAHVCHNFNVPFVVVRAISDVADQQSHLSFDEFLAVAAKQSSLMVESLVQKLAHG\n'

In [54]:
def get_alphafold_structure(pdb_id, uniprot_csv):
    uniprot_id_dict = pdb_to_uniprot(pdb_id, uniprot_csv)
    alphafold_chains_dict= {}
    for i in uniprot_id_dict.keys():
        try:
            alphafold_chains_dict[i] = PandasPdb().fetch_pdb(uniprot_id=uniprot_id_dict[i], source='alphafold2-v2')
        except AttributeError:
            alphafold_chains_dict[i] = 'Needs Carlos', uniprot_id_dict[i]
    return alphafold_chains_dict


In [55]:
def save_alphafold_structure_chain(pdb_id, uniprot_csv):
    all_structures = get_alphafold_structure(pdb_id, uniprot_csv)
    chains_that_need_carlos = []
    for chain in all_structures.keys():
        try:
            all_structures[chain].to_pdb(path=f'test_alphafold/{pdb_id}_alphafold_raw_{chain}.pdb', 
                    records=None, 
                    gz=False, 
                    append_newline=True)
        except AttributeError:
            print(f'PDB {pdb_id, chain, all_structures[chain][1]} has no structure in the AlphaFold database')
            chains_that_need_carlos.append(all_structures[chain][1])
            
    return chains_that_need_carlos

In [56]:
chains_that_need_carlos = []
for c in casf_pdbs:
    chains_that_need_carlos.extend(save_alphafold_structure_chain(c, pdb_to_uniprot_csv))


HTTP Error 404
HTTP Error 404
PDB ('1a30', 'A', 'P04585') has no structure in the AlphaFold database
PDB ('1a30', 'B', 'P04585') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
PDB ('1eby', 'A', 'P03366') has no structure in the AlphaFold database
PDB ('1eby', 'B', 'P03366') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
PDB ('1g2k', 'A', 'P04587') has no structure in the AlphaFold database
PDB ('1g2k', 'B', 'P04587') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
PDB ('2qnq', 'A', 'P03367') has no structure in the AlphaFold database
PDB ('2qnq', 'B', 'P03367') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
HTTP Error 404
HTTP Error 404
HTTP Error 404
PDB ('2wn9', 'A', 'Q8WSF8') has no structure in the AlphaFold database
PDB ('2wn9', 'B', 'Q8WSF8') has no structure in the AlphaFold database
PDB ('2wn9', 'C', 'Q8WSF8') has no structure in the AlphaFold database
PDB ('2wn9', 'D', 'Q8WSF

In [57]:
list(set(chains_that_need_carlos))

['P04585',
 'Q8A3I4',
 'Q9L5C8',
 'P03366',
 'O92972',
 'P03369',
 'O67854',
 'Q9AMP1',
 'Q8WSF8',
 'P26663',
 'Q72498',
 'P12497',
 'P04587',
 'P03367']

In [58]:
def get_chain_ids(pdb_id):
    load_crystal_receptor(pdb_id)
    chains = cmd.get_chains(f'{pdb_id}_protein_cleaned')
    cmd.delete("all")
    return chains

In [59]:
def load_crystal_ligand_pose(pdb_id):
    cmd.load(f'../pdbbind_2020_general/{pdb_id}/{pdb_id}_ligand_pymol.pdb')

In [60]:
def load_crystal_receptor(pdb_id):
    cmd.load(f'../pdbbind_2020_general/{pdb_id}/{pdb_id}_protein_cleaned.pdb')

In [61]:
def load_alphafold_receptor_chain(pdb_id, chain):
    cmd.load(f'test_alphafold/{pdb_id}_alphafold_raw_{chain}.pdb')

In [62]:
def align_protein_to_selection(pdb_id, chain):
    selection_name = create_selection_of_chain(f'{pdb_id}_protein_cleaned', chain)
    cmd.align(f'{pdb_id}_alphafold_raw_{chain}', selection_name)

In [63]:
def create_selection_of_chain(protein_name, chain):
    selection_name = chain+'_'+protein_name
    cmd.select(selection_name, protein_name+' and chain ' + chain)
    return selection_name

In [75]:
def check_sequence_lengths_equal(pdb_id):
    try:
        crystal = parser.get_structure('crystal', f'../pdbbind_2020_general/{pdb_id}/{pdb_id}_protein_cleaned.pdb')
        alphafold = parser.get_structure('alphafold', f'test_alphafold/{pdb_id}_alphafold.pdb')
        num_of_residues = []
        for stucture in crystal, alphafold:
            num_of_resides.append(get_num_of_residues(structure))
        print(pdb_id, len(list(set(num_of_residues))))
        return len(list(set(num_of_residues)))
    except:
        crystal = parser.get_structure('crystal', f'../pdbbind_2020_general/{pdb_id}/{pdb_id}_protein_cleaned.pdb')
        print(pdb_id, 'No alphafold structure yet', get_num_of_residues(crystal)/len(get_chain_ids(pdb_id)))
        return None

In [65]:
def save_aligned_structure(pdb_id, chain):
    cmd.save(f'test_alphafold/{pdb_id}_alphafold_{chain}.pdb', f'{pdb_id}_alphafold_raw_{chain}')
    

In [66]:
def align_alphafold_chain_to_pdb_chain(pdb_id):
    chains = get_chain_ids(pdb_id)
    for chain in chains: 
        load_crystal_receptor(pdb_id)
        load_alphafold_receptor_chain(pdb_id, chain)
        align_protein_to_selection(pdb_id, chain)
        save_aligned_structure(pdb_id, chain)   
    cmd.delete("all")
    return chains

In [70]:
def build_alphafold_multimer(pdb_id):
    try:
        chains = align_alphafold_chain_to_pdb_chain(pdb_id)
        string_selection = ''
        for chain in chains:
            cmd.load(f'test_alphafold/{pdb_id}_alphafold_{chain}.pdb')
            string_selection += f'+ {pdb_id}_alphafold_{chain} '
            cmd.alter(f'{pdb_id}_alphafold_{chain}', f"chain = '{chain}'") 
        cmd.select('all_chains', string_selection[2:])
        cmd.create(f'{pdb_id}_alphafold', 'all_chains')
        cmd.save(f'test_alphafold/{pdb_id}_alphafold.pdb', 'all_chains')
    except:
        print(f'{pdb_id} currently has no alphafold structure')

In [152]:
build_alphafold_multimer('1nc1')

c1_alphafold_A + 1nc1_alphafold_B 


In [69]:
for c in casf_pdbs:
    save_alphafold_structure_chain(c, pdb_to_uniprot_csv)

HTTP Error 404
HTTP Error 404
PDB ('1a30', 'A', 'P04585') has no structure in the AlphaFold database
PDB ('1a30', 'B', 'P04585') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
PDB ('1eby', 'A', 'P03366') has no structure in the AlphaFold database
PDB ('1eby', 'B', 'P03366') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
PDB ('1g2k', 'A', 'P04587') has no structure in the AlphaFold database
PDB ('1g2k', 'B', 'P04587') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
PDB ('2qnq', 'A', 'P03367') has no structure in the AlphaFold database
PDB ('2qnq', 'B', 'P03367') has no structure in the AlphaFold database
HTTP Error 404
HTTP Error 404
HTTP Error 404
HTTP Error 404
HTTP Error 404
PDB ('2wn9', 'A', 'Q8WSF8') has no structure in the AlphaFold database
PDB ('2wn9', 'B', 'Q8WSF8') has no structure in the AlphaFold database
PDB ('2wn9', 'C', 'Q8WSF8') has no structure in the AlphaFold database
PDB ('2wn9', 'D', 'Q8WSF

In [71]:
for c in casf_pdbs:
    build_alphafold_multimer(c)

1a30 currently has no alphafold structure
1e66 currently has no alphafold structure
1eby currently has no alphafold structure
1g2k currently has no alphafold structure
1p1n currently has no alphafold structure
1qkt currently has no alphafold structure
1vso currently has no alphafold structure
2qnq currently has no alphafold structure
2wn9 currently has no alphafold structure
2wnc currently has no alphafold structure
2wvt currently has no alphafold structure
2x00 currently has no alphafold structure
2xb8 currently has no alphafold structure
2xdl currently has no alphafold structure
2xii currently has no alphafold structure
2xys currently has no alphafold structure
2ymd currently has no alphafold structure
3ao4 currently has no alphafold structure
3arp currently has no alphafold structure
3arq currently has no alphafold structure
3aru currently has no alphafold structure
3arv currently has no alphafold structure
3ary currently has no alphafold structure
3cj4 currently has no alphafold st

In [76]:
for c in casf_pdbs:
    check_sequence_lengths_equal(c)

1a30 No alphafold structure yet 99.0
1bcu No alphafold structure yet 137.5
1bzc No alphafold structure yet 297.0
1c5z No alphafold structure yet 245.0
1e66 No alphafold structure yet 532.0
1eby No alphafold structure yet 99.0
1g2k No alphafold structure yet 99.0
1gpk No alphafold structure yet 529.0
1gpn No alphafold structure yet 529.0
1h22 No alphafold structure yet 529.0
1h23 No alphafold structure yet 528.0
1k1i No alphafold structure yet 223.0
1lpg No alphafold structure yet 234.0
1mq6 No alphafold structure yet 142.0
1nc1 No alphafold structure yet 233.0
1nc3 No alphafold structure yet 232.5
1nvq No alphafold structure yet 264.0
1o0h No alphafold structure yet 124.0
1o3f No alphafold structure yet 223.0
1o5b No alphafold structure yet 245.0
1owh No alphafold structure yet 245.0
1oyt No alphafold structure yet 138.5
1p1n No alphafold structure yet 258.0
1p1q No alphafold structure yet 258.0
1ps3 No alphafold structure yet 1014.0
1pxn No alphafold structure yet 294.0
1q8t No alphaf

4ddh No alphafold structure yet 284.0
4ddk No alphafold structure yet 285.0
4de1 No alphafold structure yet 262.0
4de2 No alphafold structure yet 260.0
4de3 No alphafold structure yet 260.0
4djv No alphafold structure yet 390.0
4dld No alphafold structure yet 254.5
4dli No alphafold structure yet 334.0
4e5w No alphafold structure yet 285.0
4e6q No alphafold structure yet 296.0
4ea2 No alphafold structure yet 281.0
4eky No alphafold structure yet 809.0
4eo8 No alphafold structure yet 557.0
4eor No alphafold structure yet 275.0
4f09 No alphafold structure yet 288.0
4f2w No alphafold structure yet 234.0
4f3c No alphafold structure yet 230.0
4f9w No alphafold structure yet 336.0
4gfm No alphafold structure yet 289.0
4gid No alphafold structure yet 387.0
4gkm No alphafold structure yet 346.0
4gr0 No alphafold structure yet 159.0
4hge No alphafold structure yet 284.0
4ih5 No alphafold structure yet 557.0
4ih7 No alphafold structure yet 557.0
4ivb No alphafold structure yet 291.0
4ivc No alph

In [30]:
from Bio.PDB.PDBParser import PDBParser
from Bio import PDB
parser = PDBParser(PERMISSIVE=1)

In [31]:
structure = parser.get_structure('test_alphafold', 'test_alphafold/1nc1_alphafold_A.pdb')

/data/pegasus/not-backed-up/durant/miniconda3/envs/pointvs/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1711
  warnings.warn(


In [32]:
for i in structure.get_residues():
    num_of_residues = int(str(i).split(' ')[4].split('=')[1])

In [33]:
num_of_residues

232

In [34]:
true_structure = parser.get_structure('test_pdb', '../pdbbind_2020_general/1nc1/1nc1_protein_cleaned.pdb')

In [37]:
def get_num_of_residues(biopython_structure):
    res_no = 0
    non_resi = 0
    for model in biopython_structure:
        for residue in model.get_residues():
            if PDB.is_aa(residue):
                res_no += 1

            else:
                non_resi += 1
    return res_no

In [38]:
get_num_of_residues(true_structure)

466